# Sample Hangul RNN

In [1]:
# -*- coding: utf-8 -*-
# Import Packages
import numpy as np
import tensorflow as tf
import collections
import string
import argparse
import time
import os
from six.moves import cPickle
from TextLoader import *
from Hangulpy3 import *

# Load dataset using TextLoader

In [2]:
data_dir    = "nine_dreams"
batch_size  = 50
seq_length  = 50
data_loader = TextLoader(data_dir, batch_size, seq_length)
# This makes "vocab.pkl" and "data.npy" in "nine_dreams"   
#  from "nine_dreams/input.txt" 
vocab_size = data_loader.vocab_size
vocab = data_loader.vocab
chars = data_loader.chars
print ( "type of 'data_loader' is %s, length is %d" 
       % (type(data_loader.vocab), len(data_loader.vocab)) )
print ( "\n" )
print ("data_loader.vocab looks like \n%s " %
       (data_loader.vocab))
print ( "\n" )
print ( "type of 'data_loader.chars' is %s, length is %d" 
       % (type(data_loader.chars), len(data_loader.chars)) )
print ( "\n" )
print ("data_loader.chars looks like \n%s " % (data_loader.chars,))

loading preprocessed files
type of 'data_loader' is <class 'dict'>, length is 76


data_loader.vocab looks like 
{'ㄿ': 56, 'ㅋ': 42, 'ㅘ': 26, 'ㅌ': 30, 'ㅙ': 53, '.': 24, 'ㅍ': 31, 'ㅣ': 6, 'ㅚ': 33, 'ㅁ': 12, '!': 52, 'ㅢ': 25, 'ㄱ': 7, 'ㅆ': 22, 'ㅒ': 74, '?': 41, 'ㅃ': 47, 'ᴥ': 0, '(': 45, 'ㅇ': 2, 'ㄳ': 60, 'ㅉ': 40, 'ㅖ': 39, 'ㅔ': 20, 'ㅅ': 10, 'ㅝ': 36, 'ㅂ': 17, 'ㅛ': 38, 'ㄺ': 48, 'ㅟ': 35, 'ㄷ': 13, 'ㅡ': 8, ' ': 1, '7': 71, 'ㄲ': 29, '2': 62, '"': 28, 'ㄼ': 54, 'ㅓ': 11, 'ㅀ': 51, 'ㄾ': 70, 'ㅗ': 9, 'ㄴ': 4, '\x1a': 72, ')': 46, 'ㅐ': 21, 'ㅎ': 14, 'ㄹ': 5, 'ㅊ': 23, 'ㄻ': 55, 'ㅈ': 15, '1': 58, 'ㅜ': 16, '_': 69, '0': 73, '3': 66, "'": 49, '6': 59, '4': 67, 'ㄵ': 50, '5': 63, 'ㅞ': 61, '-': 65, 'ㅕ': 18, '\n': 19, 'ㅄ': 43, 'ㅑ': 34, '9': 64, 'ㅠ': 37, 'ㄸ': 32, 'ㄶ': 44, ':': 57, '>': 75, 'ㅏ': 3, ',': 27, '8': 68} 


type of 'data_loader.chars' is <class 'tuple'>, length is 76


data_loader.chars looks like 
('ᴥ', ' ', 'ㅇ', 'ㅏ', 'ㄴ', 'ㄹ', 'ㅣ', 'ㄱ', 'ㅡ', 'ㅗ', 'ㅅ', 'ㅓ', 'ㅁ', 'ㄷ', 'ㅎ', 'ㅈ', 'ㅜ', 'ㅂ', 'ㅕ', '\n', 'ㅔ', 'ㅐ', 

# Define Network

In [3]:
rnn_size   = 512
num_layers = 3
grad_clip  = 5.

_batch_size = 1
_seq_length = 1

vocab_size = data_loader.vocab_size


# Select RNN Cell
unitcell = tf.nn.rnn_cell.BasicLSTMCell(rnn_size)
cell = tf.nn.rnn_cell.MultiRNNCell([unitcell] * num_layers)
# Set paths to the graph 
input_data = tf.placeholder(tf.int32, [_batch_size, _seq_length])
targets    = tf.placeholder(tf.int32, [_batch_size, _seq_length])
initial_state = cell.zero_state(_batch_size, tf.float32)

# Set Network
with tf.variable_scope('rnnlm'):
    softmax_w = tf.get_variable("softmax_w", [rnn_size, vocab_size])
    softmax_b = tf.get_variable("softmax_b", [vocab_size])
    with tf.device("/cpu:0"):
        embedding = tf.get_variable("embedding", [vocab_size, rnn_size])
        inputs = tf.split(1, _seq_length, tf.nn.embedding_lookup(embedding, input_data))
        inputs = [tf.squeeze(input_, [1]) for input_ in inputs]

# Loop function for seq2seq
def loop(prev, _):
    prev = tf.nn.xw_plus_b(prev, softmax_w, softmax_b)
    prev_symbol = tf.stop_gradient(tf.argmax(prev, 1))
    return tf.nn.embedding_lookup(embedding, prev_symbol)
# Output of RNN 
outputs, last_state = tf.nn.seq2seq.rnn_decoder(inputs, initial_state
                            , cell, loop_function=None, scope='rnnlm')
output = tf.reshape(tf.concat(1, outputs), [-1, rnn_size])
logits = tf.nn.xw_plus_b(output, softmax_w, softmax_b)
# Next word probability 
probs = tf.nn.softmax(logits)
# Define LOSS
loss = tf.nn.seq2seq.sequence_loss_by_example([logits], # Input
    [tf.reshape(targets, [-1])], # Target
    [tf.ones([_batch_size * _seq_length])], # Weight 
    vocab_size)
# Define Optimizer
cost = tf.reduce_sum(loss) / _batch_size / _seq_length
final_state = last_state
lr = tf.Variable(0.0, trainable=False)
tvars = tf.trainable_variables()
grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), grad_clip)
_optm = tf.train.AdamOptimizer(lr)
optm = _optm.apply_gradients(zip(grads, tvars))

print ("그래프 준비됨")

그래프 준비됨


In [4]:
# Sample ! 
def sample( sess, chars, vocab, __probs, num=200, prime=u'ㅇㅗᴥㄴㅡㄹᴥ '):
    state = sess.run(cell.zero_state(1, tf.float32))
    _probs = __probs
    prime = list(prime)
    for char in prime[:-1]:
        x = np.zeros((1, 1))
        x[0, 0] = vocab[char]
        feed = {input_data: x, initial_state:state}
        [state] = sess.run([final_state], feed)

    def weighted_pick(weights):
        weights = weights / np.sum(weights) 
        t = np.cumsum(weights)
        s = np.sum(weights)
        return(int(np.searchsorted(t, np.random.rand(1)*s)))

    ret = prime
    char = prime[-1]
    for n in range(num):
        x = np.zeros((1, 1))
        x[0, 0] = vocab[char]
        feed = {input_data: x, initial_state:state}
        [_probsval, state] = sess.run([_probs, final_state], feed)
        p = _probsval[0]
        sample = int(np.random.choice(len(p), p=p))
        # sample = weighted_pick(p)
        # sample = np.argmax(p)
        pred = chars[sample]
        ret += pred
        char = pred
    return ret
print ("샘플링 함수 완료.")

샘플링 함수 완료.


# Sample

In [5]:
save_dir = 'data/nine_dreams'
prime = decompose_text("누구 ")

print ("Prime Text : %s => %s" % (automata(prime), "".join(prime)))
n = 2000

# 세션 설정
session_conf = tf.ConfigProto()
session_conf.gpu_options.allow_growth = True

sess = tf.Session(config=session_conf)
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(tf.global_variables())
ckpt = tf.train.get_checkpoint_state(save_dir)

# load_name = 'nine_dreams/model.ckpt-0'
# load_name = 'nine_dreams/model.ckpt-99000'
load_name = 'nine_dreams/model.ckpt-4000'

print (load_name)

saver.restore(sess, load_name)
sampled_text = sample(sess, chars, vocab, probs, n, prime)
#print ("")
print ("예시 문장 = %s" % sampled_text)
print ("")
print ("-- 결과 --")
print (automata("".join(sampled_text)))

Prime Text : 누구  => ㄴㅜᴥㄱㅜᴥ 
nine_dreams/model.ckpt-4000
예시 문장 = ['ㄴ', 'ㅜ', 'ᴥ', 'ㄱ', 'ㅜ', 'ᴥ', ' ', 'ㅇ', 'ㅣ', 'ㄴ', 'ᴥ', 'ㅂ', 'ㅏ', 'ㄲ', 'ᴥ', ' ', 'ㅇ', 'ㅣ', 'ᴥ', 'ㅇ', 'ㅟ', 'ᴥ', 'ㅇ', 'ㅔ', 'ᴥ', ' ', 'ㄷ', 'ㅏ', 'ᴥ', 'ㅅ', 'ㅣ', 'ᴥ', ' ', 'ㅈ', 'ㅜ', 'ᴥ', 'ㅁ', 'ㅗ', 'ㄴ', 'ᴥ', ' ', 'ㄷ', 'ㅡ', 'ㄹ', 'ᴥ', 'ㅇ', 'ㅗ', 'ㄴ', 'ᴥ', 'ㅇ', 'ㅡ', 'ㄴ', 'ᴥ', ' ', 'ㅁ', 'ㅏ', 'ㄴ', 'ᴥ', 'ㅇ', 'ㅣ', 'ᴥ', ' ', 'ㅊ', 'ㅣ', 'ㄴ', 'ᴥ', 'ㄱ', 'ㅜ', 'ᴥ', 'ㅎ', 'ㅏ', 'ㅁ', 'ᴥ', 'ㄷ', 'ㅣ', 'ㄴ', 'ᴥ', ' ', 'ㅁ', 'ㅏ', 'ᴥ', 'ㅇ', 'ㅕ', 'ᴥ', ' ', 'ㅇ', 'ㅣ', 'ᴥ', 'ㅇ', 'ㅓ', 'ᴥ', 'ㅅ', 'ㅣ', 'ᴥ', 'ㄱ', 'ㅣ', 'ᴥ', ' ', 'ㄱ', 'ㅏ', 'ㅌ', 'ᴥ', 'ㅇ', 'ㅣ', 'ᴥ', ' ', 'ㅇ', 'ㅣ', 'ㅍ', 'ᴥ', 'ㄸ', 'ㅣ', 'ᴥ', ' ', '\n', 'ㄱ', 'ㅏ', 'ㅌ', 'ᴥ', 'ㅇ', 'ㅣ', 'ᴥ', ' ', 'ㄸ', 'ㅗ', 'ᴥ', 'ㅎ', 'ㅏ', 'ㄴ', 'ᴥ', 'ㄴ', 'ㅡ', 'ㄴ', 'ᴥ', ' ', 'ㄱ', 'ㅡ', 'ᴥ', ' ', 'ㅇ', 'ㅣ', 'ㄹ', 'ᴥ', 'ㅇ', 'ㅡ', 'ᴥ', 'ㄹ', 'ㅣ', 'ᴥ', 'ㄹ', 'ㅡ', 'ㄹ', 'ᴥ', ' ', 'ㅃ', 'ㅓ', 'ㄴ', 'ᴥ', 'ㅈ', 'ㅏ', 'ㄱ', 'ᴥ', 'ㄱ', 'ㅏ', 'ㅁ', 'ᴥ', ' ', 'ㄱ', 'ㅠ', 'ㅇ', 'ᴥ', 'ㄱ', 'ㅕ', 'ㅇ', 'ᴥ', 'ㅅ', 'ㅣ', 'ᴥ', 'ㄹ', 'ㅕ', 'ᴥ', 'ㅂ', 'ㅏ', 'ㅂ', 'ᴥ', 'ㅇ', 'ㅡ',